In [11]:
using Pkg
Pkg.add(["Clustering", "StatsPlots", "DataFrames", "CSV"])


  Resolving package versions...
  Installed CSV ─ v0.8.4
   Updating `~/.julia/environments/v1.4/Project.toml`
  [336ed68f] + CSV v0.8.4
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [336ed68f] + CSV v0.8.4


In [136]:
using Distributed, Random, Clustering, StatsPlots, DataFrames, LinearAlgebra
@everywhere using CSV, SharedArrays

#The same as the Python one, uses arbitrary precision
@everywhere function dist_quakes(sismo_i, sismo_j, c, B)
 tau = abs(sismo_j[5] - sismo_i[5])
 lat1 = sismo_i[2]
 lat2 = sismo_j[2]
 lon1 =  sismo_i[3]
 lon2 = sismo_j[3]

 rad = pi/180
 dlat = lat2-lat1
 dlon = lon2-lon1
 R = 6372.795477598
 a = (sin(rad*dlat/2))^2 + cos(rad*lat1)*cos(rad*lat2)*(sin(rad*dlon/2))^2
 distancia = 2*R*asin(sqrt(a))

 r = sqrt(distancia^2 + (sismo_j[4]-sismo_i[4])^2 )
 delta_m = convert(BigFloat, sismo_j[1] - sismo_i[1])
 return c*tau*(r^(2*B) )* (10^(-B * delta_m))
end


traindf = CSV.read("train.csv", DataFrame)
#To do parallel processing
train = SharedArray(Matrix(train))

#Only the upper triangular part will be filled
distances = zeros(BigFloat, 6000, 6000)
@sync @distributed for i = 1:6000
  for j = (i+1):6000
    distances[i,j] = dist_quakes(train[i,:], train[j,:], 1, 0.6160988624731996)
  end
end

distances = LinearAlgebra.Symmetric(distances, :U)
distances = Matrix{Float64}(distances)
CSV.write("distances.csv", DataFrame(distances, :auto))

### Clustering

Random.seed!(3)

tree = hclust(distances, linkage=:ward)
plot(tree, branchorder=:optimal)
savefig("dendogram2_6000.pdf")
#Too many leaves

tree_1000 = hclust(distances[1:1000, 1:1000], linkage=:ward)
plot(tree_1000, branchorder=:optimal)
savefig("dendogram2_1000.pdf")
#Still too many

tree_100 = hclust(distances[1:100, 1:100], linkage=:ward)
plot(tree_100, branchorder=:optimal)
savefig("dendogram2_100.pdf")

# perhaps 3 clusters?


In [170]:
clust = cutree(tree, h=50)

df6000 = DataFrame(reshape(clust, 6000, 1), :auto)

ttrain = traindf[1:6000, :]

ttrain[!, "Cluster"] = df6000[!, "x1"]

ttrain

,Column1,0,1,2,3,4,Cluster
,Int64,Float64,Float64,Float64,Float64,Float64,Int64
1,0,3.4,15.9832,-96.9505,25.8,0.0,1
2,1,3.5,16.6938,-95.1257,94.8,211.0,2
3,2,3.8,15.5425,-92.4787,187.6,2679.0,3
4,3,3.5,16.0508,-97.1588,26.3,2952.0,4
5,4,3.6,17.173,-94.979,121.0,3482.0,5
6,5,4.3,15.782,-94.9405,16.1,7140.0,2
7,6,3.6,17.3505,-94.5152,136.2,8449.0,6
8,7,3.7,16.3592,-93.7075,64.4,8628.0,7
9,8,3.6,16.2487,-95.7268,47.4,9368.0,2


In [173]:
CSV.write("clust6000.csv", ttrain, header=true)

"clust6000.csv"